In [1]:
from the2 import *

In [2]:
forward_pass = ForwardPass()

# Test 1
forward_pass(
    gergen([[1], [2], [3], [4]]),
    gergen([[1,2, 3,4]]),
    gergen([[100, 100, 100, 100], [200, 200, 200, 200], [300, 300, 300, 300], [400, 400, 400, 400]])
)

In [3]:
main()

chp1
chp2
x: 12x1 boyutlu gergen:
[[145.77005144119036], [0], [0], [83.63475716090758], [3.1113443635673983], [0], [0], [0], [51.02600506426839], [59.32246363482439], [0], [251.88339333796864]]
 W: 10x12 boyutlu gergen:
[[0.09194369709966152, -0.00041847573618700173, -0.009834340974858621, 0.04687232702215341, 0.07311162691783252, 0.08471242611723065, 0.00851516269992543, 0.07906412041272842, 0.09896546062615086, -0.08924692106741841, 0.08975988560527462, -0.07752925406314337], [-0.07840944920460752, 0.042142375292014894, 0.0866416231720285, 0.03542968835945068, -0.036168045843167934, -0.06979950669373458, 0.08735049265099099, 0.02225468298182634, 0.0839793913256984, -0.017002238473620174, -0.03713837740798202, -0.0562583878801167], [-0.0511895289168651, -0.06706843409560348, -0.0341691972199637, -0.02292145914631613, 0.06528853607121238, -0.0841816253074931, -0.046794580905219554, 0.06864150479416897, 0.07993465513688311, -0.07625122525277446, 0.07201035853475521, -0.01134653220655139

OverflowError: math range error